# TVA sur les biens alimentaires

In [1]:
from __future__ import division

import os
import pandas as pd
import pkg_resources
import seaborn

from openfisca_france_indirect_taxation.surveys import SurveyScenario

In [42]:
year = 2014
data_year = 2011
survey_scenario = SurveyScenario.create(year = year, data_year = 2011, reform_key = 'aliss_tva_sociale')

In [52]:
alimentation_domicile_hors_alcool = [
    "depenses_ht_{}".format(key) for key in survey_scenario.tax_benefit_system.column_by_name.keys() \
    if key.startswith(u'poste_01')
    ]
alimentation_domicile = alimentation_domicile_hors_alcool + [
    'depenses_biere',
    'depenses_vin',
    'depenses_alcools_forts'
    ]
alimentation_domicile

['depenses_ht_poste_01_1_1_1_1',
 'depenses_ht_poste_01_1_1_3_3',
 'depenses_ht_poste_01_1_1_4_1',
 'depenses_ht_poste_01_1_1_4_2',
 'depenses_ht_poste_01_1_1_4_3',
 'depenses_ht_poste_01_1_2_1_1',
 'depenses_ht_poste_01_1_2_3_1',
 'depenses_ht_poste_01_1_2_4_1',
 'depenses_ht_poste_01_1_2_4_2',
 'depenses_ht_poste_01_1_2_5_1',
 'depenses_ht_poste_01_1_2_6_1',
 'depenses_ht_poste_01_1_2_6_3',
 'depenses_ht_poste_01_1_3_1_1',
 'depenses_ht_poste_01_1_3_1_2',
 'depenses_ht_poste_01_1_3_2_1_a',
 'depenses_ht_poste_01_1_3_2_1_b',
 'depenses_ht_poste_01_1_3_2_2',
 'depenses_ht_poste_01_1_4_1_1_a',
 'depenses_ht_poste_01_1_4_1_1_b',
 'depenses_ht_poste_01_1_4_1_1_c',
 'depenses_ht_poste_01_1_4_2_1',
 'depenses_ht_poste_01_1_4_2_2',
 'depenses_ht_poste_01_1_4_3_1',
 'depenses_ht_poste_01_1_4_4_1',
 'depenses_ht_poste_01_1_5_1_1',
 'depenses_ht_poste_01_1_5_2_1_a',
 'depenses_ht_poste_01_1_5_2_1_b',
 'depenses_ht_poste_01_1_5_2_2_a',
 'depenses_ht_poste_01_1_5_2_2_b',
 'depenses_ht_poste_01_1_

In [46]:
variables = alimentation_domicile + [
    'poste_agrege_01',
    'poste_agrege_02', 
    'depenses_ht_tva_taux_super_reduit',
    'depenses_ht_tva_taux_reduit',
    'depenses_ht_tva_taux_intermediaire',
    'depenses_ht_tva_taux_plein',
    'tva_taux_super_reduit',
    'tva_taux_reduit',
    'tva_taux_intermediaire',
    'tva_taux_plein',aliss_tva_sociale
    'tva_total',
    ]

In [60]:
pivot_table = pd.DataFrame()
pivot_table_reference = pd.DataFrame()
aggregates = dict()
reference_aggregates = dict()
for variable in variables:
    pivot_table = pd.concat([
        pivot_table,
        survey_scenario.compute_pivot_table(values = [variable], columns = ['niveau_vie_decile'])
        ])
    pivot_table_reference = pd.concat([
        pivot_table_reference,
        survey_scenario.compute_pivot_table(values = [variable], columns = ['niveau_vie_decile'],
                                           reference = True)
        ])
    aggregates[variable] = survey_scenario.compute_aggregate(variable) / 1e9
    reference_aggregates[variable] = survey_scenario.compute_aggregate(variable, reference = True) / 1e9

In [61]:
assert sum(aggregates[aliment] - reference_aggregates[aliment] for aliment in alimentation_domicile) == 0

In [62]:
assert aggregates['poste_agrege_02'] - reference_aggregates['poste_agrege_02'] < 1e-7

In [63]:
for key in sorted(aggregates.keys()):
    if round(aggregates[key] - reference_aggregates[key], 3) != 0:
        print key, ': ', round(aggregates[key], 3), ' - ', round(reference_aggregates[key], 3), ' = ', round(aggregates[key] - reference_aggregates[key], 3)

depenses_ht_tva_taux_intermediaire :  183.132  -  75.594  =  107.538
depenses_ht_tva_taux_plein :  311.769  -  315.773  =  -4.004
depenses_ht_tva_taux_reduit :  24.994  -  128.528  =  -103.534
poste_agrege_01 :  128.399  -  124.14  =  4.259
tva_taux_intermediaire :  18.313  -  7.559  =  10.754
tva_taux_plein :  62.354  -  63.155  =  -0.801
tva_taux_reduit :  1.375  -  7.069  =  -5.694
tva_total :  82.183  -  77.924  =  4.259


In [64]:
from openfisca_france_indirect_taxation.model.consommation.postes_coicop import get_legislation_data_frames
categories_fiscales_data_frame, codes_coicop_data_frame = get_legislation_data_frames()

In [65]:
aggregates['poste_agrege_01'] - reference_aggregates['poste_agrege_01']

4.2585982890594494

In [67]:
pivot_table -pivot_table_reference

niveau_vie_decile,1,2,3,4,5,6,7,8,9,10
depenses_ht_poste_01_1_1_1_1,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
depenses_ht_poste_01_1_1_3_3,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
depenses_ht_poste_01_1_1_4_1,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
depenses_ht_poste_01_1_1_4_2,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
depenses_ht_poste_01_1_1_4_3,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
depenses_ht_poste_01_1_2_1_1,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
depenses_ht_poste_01_1_2_3_1,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
depenses_ht_poste_01_1_2_4_1,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
depenses_ht_poste_01_1_2_4_2,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
depenses_ht_poste_01_1_2_5_1,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
